In [1]:
from __future__ import print_function
import os
import neat

import pandas as pd
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim


from explaneat.core.backprop import NeatNet
from explaneat.core import backprop
from explaneat.core.backproppop import BackpropPopulation
from explaneat.visualization import visualize
from explaneat.core.experiment import ExperimentReporter
from explaneat.core.utility import one_hot_encode


from sklearn import datasets
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


from copy import deepcopy

import time
from datetime import datetime


import gzip
try:
    import cPickle as pickle  # pylint: disable=import-error
except ImportError:
    import pickle  # pylint: disable=import-error

In [2]:

USE_CUDA = torch.cuda.is_available()
USE_CUDA = False
device = torch.device("cuda:1" if USE_CUDA else "cpu")


# Breast Cancer Experiment

This experiment (a) test the experimental environment, but is also to evaluate the efficacy of the ExplaNEAT algorithm. Speed is a critical factor, as well as stability of results on population size. Total run time will also be measured

First, we need to set a random seed and a total stopping point in the number of generations

In [3]:
my_random_seed = 42
random.seed(my_random_seed)

In [4]:
def one_hot_encode(vals):
    width = max(vals)
    newVals = []
    for val in vals:
        blank = [0. for _ in range(width + 1)]
        blank[val] = 1.
        newVals.append(blank)
    return np.asarray(newVals)


## Dataset

We are going to work with the Iris dataset, which will be loaded from `sklearn`. We want to characterise the efficacy of the algorithm with regards to a mostly untransformed dataset, so we will only normalise the features

In [5]:
breast_cancer = datasets.load_breast_cancer()
xs_raw = breast_cancer.data[:, :]
scaler = StandardScaler()
scaler.fit(xs_raw)
xs = scaler.transform(xs_raw)
ys = breast_cancer.target.astype(np.float32)
# ys_onehot = one_hot_encode(ys)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(xs, ys, test_size=69, random_state=42)

In [7]:
# xs = torch.from_numpy(xs).to(device)
# ys = torch.from_numpy(ys).to(device)
X_train = torch.from_numpy(X_train).to(device)
X_test = torch.from_numpy(X_test).to(device)
y_train = torch.from_numpy(y_train).to(device)
y_test = torch.from_numpy(y_test).to(device)

In [8]:
xs = torch.from_numpy(xs).to(device)
ys = torch.from_numpy(ys).to(device)

In [ ]:
xs.shape

torch.Size([569, 30])

Let's have a look at the data we are working with

In [ ]:
ys[18:23]

tensor([0., 1., 1., 1., 0.])

In [ ]:
xs[18:20]

tensor([[ 1.6140,  0.6656,  1.5665,  1.7210,  0.1388, -0.0311,  0.7420,  1.1881,
         -0.8383, -1.2542,  1.2742, -0.3626,  1.4846,  1.5855, -0.1823, -0.3660,
          0.0669,  0.5538, -0.8454, -0.6801,  2.2884,  0.8472,  2.3691,  2.6675,
          0.8255,  0.3864,  1.2714,  1.8910, -0.2148, -0.4320],
        [-0.1668, -1.1472, -0.1857, -0.2520,  0.1017, -0.4369, -0.2782, -0.0286,
          0.2679, -0.7283, -0.4882, -0.7770, -0.4000, -0.3691,  0.4737, -0.6080,
         -0.2660,  0.2196, -0.0899, -0.5654, -0.2400, -1.0450, -0.2252, -0.2978,
          0.5099, -0.4896, -0.1592,  0.2161,  0.1233, -0.6293]],
       dtype=torch.float64)

## Performance metric

The NEAT implementation on which ExplaNEAT extends uses a single function call for evaluating fitness. Although this might be reworked for ExplaNEAT to be able to get consistency between the genome-evaluation and the backprop loss function, that can be reviewed later.

This use `Binary Cross Entropy Loss` from `PyTorch`

In [ ]:
def eval_genomes(genomes, config):
    loss = nn.BCELoss()
    loss = loss.to(device)
    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        preds = []
        for xi in X_train:
            preds.append(net.activate(xi))
        genome.fitness = float(1./loss(torch.tensor(preds).to(device), torch.tensor(y_train)))

## Base configuration

We are going to create the base configuration according to an external configuration file. Per experiment, we will adjust this, later, but this defines the defaults across all runs.

In [ ]:
config_path = "./config-breast-cancer"
base_config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)


In [ ]:
base_config.pop_size

50

We also want to put a hard limit on how long this can go on for.

In [ ]:
maxNGenerations = 200

We will create a method to manage the instantiation of a population on the basis of a specific config.

In [ ]:
def instantiate_population(config, xs, ys, saveLocation):

    if not os.path.exists(saveLocation):
        os.makedirs(saveLocation)
        
    config.save(os.path.join(saveLocation, 'config.conf'))

    # Create the population, which is the top-level object for a NEAT run.
    p = BackpropPopulation(config, 
                            xs, 
                            ys, 
                            criterion=nn.BCELoss())

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    p.add_reporter(neat.Checkpointer(5, filename_prefix=str(saveLocation) + "checkpoint-" ))
    bpReporter = backprop.BackpropReporter(True)
    p.add_reporter(bpReporter)
    p.add_reporter(ExperimentReporter(saveLocation))
    
    return p

# Experiment 1: Vary population size

The first experiment is going to examine the difference in run time different population sizes. 

In [ ]:
# epoch_points = [10, 25, 50, 100, 150]
epoch_points = [10]

In [ ]:
base_config.pop_size

50

In [ ]:
saveLocationTemplate = './../../data/experiments/breast-cancer/experiment-longepochsttsplit-{}-{}/'

## Start the experiment

In [ ]:
for epochs in epoch_points:
    for iteration_no in range(20):
        my_random_seed += 1
        random.seed(my_random_seed)
        start_time = datetime.now()
        
        print("################################################")
        print("################################################")
        print("Starting epochs {} iteration {}".format(epochs, iteration_no))
        print("Started at {}".format(start_time.strftime("%m/%d/%Y, %H:%M:%S")))
        print("################################################")
        print("################################################")
        
        
        config = deepcopy(base_config)
        
        saveLocation = saveLocationTemplate.format(epochs, iteration_no)
        
        p = instantiate_population(config, xs, ys, saveLocation)
        # Run for up to nGenerations generations.
        winner = p.run(eval_genomes, maxNGenerations, nEpochs = epochs)
        
        g = p.best_genome

        
        end_time = datetime.now()
        
        p.reporters.reporters[2].save_checkpoint(p.config, p.population, p.species, str(p.generation) + "-final")  
        
        winner_net = neat.nn.FeedForwardNetwork.create(winner, config)

        results = []
        for xi, xo in zip(xs, ys):
            output = winner_net.activate(xi)
            results.append([xi, xo, output])

        df = pd.DataFrame(results)
        df.to_csv(os.path.join(saveLocation, 'results.csv'))
        
        ancestry = p.reporters.reporters[3].trace_ancestry_of_species(g.key, p.reproduction.ancestors) 

        ancestors = {
            k: v['genome'] for k, v in p.reporters.reporters[3].ancestry.items()
        }
        
        
        
        ## Save all of these to disc
        filename = 'fullStatus.xplnt'
        print("Saving checkpoint to {0}".format(filename))

        with gzip.open(os.path.join(saveLocation, filename), 'w', compresslevel=5) as f:
            data = (p, g, ancestry, ancestors, random.getstate())
            pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)
    
        with gzip.open(os.path.join(saveLocation, 'train_test_data.pkl'), 'w', compresslevel=5) as f:
            train_Test = (X_train, X_test, y_train, y_test)
            pickle.dump(train_Test, f, protocol=pickle.HIGHEST_PROTOCOL)
#         visualize.create_ancestry_video(p.config, 
#                                         g, 
#                                         ancestry, 
#                                         ancestors, 
#                                         p.reporters.reporters[1], 
#                                         pathname=saveLocation)
        print("################################################")
        print("################################################")
        print("Have finished epochs {} iteration {}".format(epochs, iteration_no))
        print("Started at {}".format(start_time.strftime("%m/%d/%Y, %H:%M:%S")))
        print("The time is {}".format(end_time.strftime("%m/%d/%Y, %H:%M:%S")))
        print("################################################")
        print("################################################")
    

################################################
################################################
Starting epochs 10 iteration 0
Started at 08/26/2019, 03:09:30
################################################
################################################

 ****** Running generation 0 ****** 

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(1.2379, grad_fn=<DivBackward0>)


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if __name__ == '__main__':
/opt/conda/lib/python3.6/site-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([500])) that is different to the input size (torch.Size([500, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Population's average fitness: 0.19568 stdev: 0.17162
Best fitness: 0.81429 - size: (1, 30) - species 2 - id 48
ending generation %s
Average adjusted fitness: 0.121
Mean genetic distance 3.863, standard deviation 1.804
Population of 51 members in 5 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0    13      0.3    0.144     0
     2    0    16      0.8    0.193     0
     3    0    14      0.6    0.138     0
     4    0     5      0.2    0.075     0
     5    0     3      0.1    0.057     0
Total extinctions: 0
Generation time: 175.188 sec

 ****** Running generation 1 ****** 

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.6263, grad_fn=<DivBackward0>)
Population's average fitness: 0.41808 stdev: 0.31887
Best fitness: 1.58815 - size: (1, 29) - species 2 - id 80


 SPECIES TOPOLOGY IMPROVEMENT


{'genome': <neat.genome.DefaultGenome object at 0x7ff80b2832b0>, 'fitness': 1.588151931762

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.3431, grad_fn=<DivBackward0>)
Population's average fitness: 1.44177 stdev: 0.68346
Best fitness: 2.81225 - size: (2, 28) - species 2 - id 198


 SPECIES TOPOLOGY IMPROVEMENT


{'genome': <neat.genome.DefaultGenome object at 0x7ff80b0c1828>, 'fitness': 2.812246799468994, 'firstDerivatives': [0.0, 0.773859977722168, 0.4323604106903076, 0.3629601001739502, 0.428774356842041], 'secondDerivatives': [0.0, 0.773859977722168, -0.34149956703186035, -0.06940031051635742, 0.06581425666809082]}
Key: 198
Fitness: 2.812246799468994
Nodes:
	0 DefaultNodeGene(key=0, bias=0.06248359754681587, response=1.0, activation=sigmoid, aggregation=sum)
	86 DefaultNodeGene(key=86, bias=0.35203060507774353, response=1.0, activation=sigmoid, aggregation=sum)
Connections:
	DefaultConnectionGene(key=(-30, 0), weight=0.1294487863779068, enabled=False)
	DefaultConnectionGene(key=(-30, 86), weight=1.0, enabled=True)
	DefaultCo

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.2575, grad_fn=<DivBackward0>)
Population's average fitness: 1.97955 stdev: 0.82644
Best fitness: 4.08898 - size: (2, 27) - species 2 - id 295
ending generation %s
Average adjusted fitness: 0.302
Mean genetic distance 3.303, standard deviation 1.694
Population of 50 members in 5 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    6    12      1.8    0.374     0
     2    6    20      4.1    0.651     0
     3    6    14      2.7    0.447     0
     4    6     2      0.3    0.038     3
     5    6     2      0.1    0.000     5
Total extinctions: 0
Generation time: 176.008 sec (175.583 average)

 ****** Running generation 7 ****** 

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.2366, grad_fn=<DivBackward0>)
Population's average fitness: 2.25192 stdev: 1.06138
Best fitness: 4.26916 - size: (1, 24) 

...

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.0911, grad_fn=<DivBackward0>)
Population's average fitness: 6.37472 stdev: 3.20810
Best fitness: 10.79072 - size: (4, 29) - species 2 - id 1502
ending generation %s
Average adjusted fitness: 0.492
Mean genetic distance 2.822, standard deviation 1.636
Population of 49 members in 3 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   36     8      3.6    0.237     1
     2   36    23     10.8    0.744     1
     3   36    18     10.0    0.495     0
Total extinctions: 0
Generation time: 193.913 sec (207.974 average)
Saving checkpoint to ./../../data/experiments/breast-cancer/experiment-longepochsttsplit-10-0/checkpoint-36

 ****** Running generation 37 ****** 

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.0873, grad_fn=<DivBackward0>)
Population's average fitness: 6.63040 stdev: 3.61601
Best fitnes

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.0873, grad_fn=<DivBackward0>)
Population's average fitness: 7.49580 stdev: 3.27693
Best fitness: 11.15013 - size: (5, 30) - species 2 - id 1627
ending generation %s
Average adjusted fitness: 0.595
Mean genetic distance 2.608, standard deviation 1.568
Population of 49 members in 3 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   38     8      3.6    0.272     1
     2   38    21     11.2    0.785     0
     3   38    20     10.6    0.727     1
Total extinctions: 0
Generation time: 195.393 sec (206.316 average)
Saving checkpoint to ./../../data/experiments/breast-cancer/experiment-longepochsttsplit-10-0/checkpoint-38

 ****** Running generation 39 ****** 

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.0873, grad_fn=<DivBackward0>)
Population's average fitness: 7.17722 stdev: 3.38284
Best fitnes

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.0868, grad_fn=<DivBackward0>)
Population's average fitness: 6.58409 stdev: 3.71602
Best fitness: 12.72985 - size: (5, 29) - species 2 - id 1768
ending generation %s
Average adjusted fitness: 0.456
Mean genetic distance 2.645, standard deviation 1.560
Population of 50 members in 3 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   41     9      3.6    0.232     4
     2   41    22     12.7    0.648     0
     3   41    19     10.9    0.489     0
Total extinctions: 0
Generation time: 196.221 sec (200.534 average)

 ****** Running generation 42 ****** 

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.0868, grad_fn=<DivBackward0>)
Population's average fitness: 6.82431 stdev: 4.06083
Best fitness: 12.72985 - size: (5, 29) - species 2 - id 1768
ending generation %s
Average adjusted fitness: 0.483
Mean 

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.0861, grad_fn=<DivBackward0>)
Population's average fitness: 7.79372 stdev: 3.76101
Best fitness: 13.67071 - size: (5, 30) - species 2 - id 1942


 SPECIES TOPOLOGY IMPROVEMENT


{'genome': <neat.genome.DefaultGenome object at 0x7ff8062c94a8>, 'fitness': 13.670708656311035, 'firstDerivatives': [0.0, 0.773859977722168, 0.4323604106903076, 0.3629601001739502, 0.428774356842041, 0.6823399066925049, 0.5943920612335205, 0.18018007278442383, 0.0, 0.7265758514404297, 0.3369779586791992, 0.888883113861084, 1.7079830169677734, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.231292724609375, 0.5249004364013672, 0.0, 0.0, 0.1238555908203125, 0.18157577514648438, 0.0, 0.0, 0.22942256927490234, 0.0, 0.7809638977050781, 0.0, 0.40195465087890625, 0.01564788818359375, 0.1372222900390625, 0.013875961303710938, 0.2204265594482422, 0.0, 0.2573738098144531, 0.10203742980957031, 0.0, 1.3079509735107422, 0.27176570892333984, 0.0, 

mean improvement: 0.0
best improvement: tensor(0., grad_fn=<SubBackward0>)
best loss: tensor(0.0861, grad_fn=<DivBackward0>)
Population's average fitness: 7.83704 stdev: 3.94810
Best fitness: 13.67071 - size: (5, 30) - species 2 - id 1942
ending generation %s
Average adjusted fitness: 0.478
Mean genetic distance 2.599, standard deviation 1.641
Population of 50 members in 3 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1   46     8      3.6    0.209     9
     2   46    23     13.7    0.676     1
     3   46    19     10.9    0.548     1
Total extinctions: 0
Generation time: 219.852 sec (205.292 average)
Saving checkpoint to ./../../data/experiments/breast-cancer/experiment-longepochsttsplit-10-0/checkpoint-46

 ****** Running generation 47 ****** 

